In [11]:
##############################################################################################################################
###         Hassan Shahzad
###         CS-D
###         Artificial Intelligence Lab (Assignment # 1)
###         FAST-NUCES
###         chhxnshah@gmail.com
##############################################################################################################################

from copy import deepcopy                        # Will be used to deep copy
import random                                    # Will be used for randomization
from math import exp                             # will be used for probability calculation
import numpy as np
board_size =0
Temp = 50000

#################################################### UTILITY FUNCTIONS #######################################################

def printBoard(board):                           # Used to print board
    for i in range(board_size):
        for j in range(board_size):
            print(board[i][j], end=' ')
        print()

def Temperature(t):                              # Will be used in Simulated Annealing  
    t+=1
    return t
        
        
##############################################################################################################################

#################################################### QUEEN FUNCTIONS #######################################################

def placeQueens(board):                          # This function places queen randomly at each column
    i = 0
    while (i < board_size):
        row = random.randint(0, board_size - 1)
        if (board[row][i] != "Q"):
            board[row][i] = "Q"
            i+=1

def getQueens(board):                            # This will get the positions of queens palced in the maze
    queen_positions = []                         # Will store list of touples of queen's positions
    for i in range(board_size):
        for j in range (board_size):
            if board[i][j] == "Q":
                temp = i,j
                queen_positions.append(temp)
    return queen_positions


def objective_function(board):                   # This function will return the number of attacking queens
    positions = getQueens(board)
    attacking_queens = 0
    count = 0
    for i in range (0,len(positions)):
        current_queen = positions[i]
        
        for j in range (0,len(positions)):
            count+=1
            if i!=j:
                other_queen = positions[j]
                
                # Step 1: Seeing if two queens are in same rows
                
                if (other_queen[0] == current_queen[0]):
                    attacking_queens+=1

                # Step 2 : Now we will check diagonals
                
                # There are a total of 4 diagonals

                otherx = other_queen[0]
                othery = other_queen[1]
                x = deepcopy(current_queen[0])            # Copying indexes
                y = deepcopy(current_queen[1])

                # Top-Left Diagonal
                
                while x >=0 and y >= 0:
                    x-=1
                    y-=1

                    if x == otherx and y == othery:
                        attacking_queens+=1
                
                        
                x = deepcopy(current_queen[0])            # Copying indexes
                y = deepcopy(current_queen[1])

                # Bottom-Right Diagonal
                
                while x <board_size and y <board_size:
                    x+=1
                    y+=1

                    if (x == otherx and y == othery):
                        attacking_queens+=1
                
                
                x = deepcopy(current_queen[0])            # Copying indexes
                y = deepcopy(current_queen[1])

                # Bottom-Left Diagonal
                
                while x <board_size and y >=0:
                    x+=1
                    y-=1

                    if (x == otherx and y == othery):
                        attacking_queens+=1
                        
                        
                x = deepcopy(current_queen[0])            # Copying indexes
                y = deepcopy(current_queen[1])        
                
                # Top-Right Diagonal
                
                while x >=0 and y <board_size:
                    x-=1
                    y+=1

                    if x == otherx and y == othery:
                        attacking_queens+=1

    return (int)(attacking_queens/2)
                    

def expandBoard (board, queen_position):                  # This function performs all possible operations
    boards = []
    
    for i in range(len(queen_position)):
        x = deepcopy(queen_position[i][0])
        y = deepcopy(queen_position[i][1])
        
        for j in range (board_size):
            if (x!=j):
                    b = deepcopy(board)
                    b[x][y] = "*"
                    b[j][y] = "Q"
                    boards.append(b)
    return boards
    
def minBoard(boards):                                     # Return the minimum heuristic value placement
    
    boards.sort(key=objective_function)
    return boards[0]


##########################################################################################################################
#################################################### HILL CLIMBING #######################################################
##########################################################################################################################

    
def hillClimbing(board):                                        # Steepest Ascend
    moves = 0
    
    while True:
        moves+=1
        queen_position = getQueens(board)                       # Getting the positions of queens
        heuristic = objective_function(board)                   # Getting number of atttacking queens
        
        boards = expandBoard(board,queen_position)              # Exploring all possible combinations
        
        minimum = minBoard(boards)                              # Returning the board with minimum number of attacking queens
        minimum_atacking_queens = objective_function(minimum)   # Returning the minimum number of attacking queens
        
        if (minimum_atacking_queens >= heuristic):
            return board, moves, heuristic
        
        elif minimum_atacking_queens == 0:                      # Finding the optimal solution (i.e No attacking queens)
            return minimum, moves, minimum_atacking_queens
        else:
            board = minimum                                     # Keep the loop iterating
                
##############################################################################################################################


##########################################################################################################################
################################################ SIMULATED ANNEALING #####################################################
##########################################################################################################################

def makeRandomMovement(board,row,col,queen_position):
    queen_position = getQueens(board)
    for r,c in queen_position:
       
        if(col==c):
            while r==row:
                row= random.randint(0,len(board)-1)
            board[row][c]=board[r][c]
            board[r][c]="*"
            
            
                
    return board
    
def simulatedAnnealing(board):                                  
    
    
    flag = False
    moves1 = 0
    current_board = board
    queen_position = getQueens(board)
    current_cost = objective_function(current_board)                # Getting current cost (heuristic)
    boards = expandBoard(current_board,queen_position)
        
    t = 0
    
    while t < Temp:
        t = Temperature(t)
        moves1 += 1
        ranrow = random.randint(0,len(board)-1)  
        rancol = random.randint(0,len(board)-1) 
        board = makeRandomMovement(board,ranrow,rancol,queen_position)
        # Randomly picking neigbours
        neighbour =deepcopy(board)                                   # Assigning any random neigbour
        
        neighbour_cost = objective_function(neighbour)             # Calculating the cost for new neighbor
        E = neighbour_cost - current_cost                          # E = current_cost - neigbour_cost (How much better neigbour is from current)
        e = (-E)/t                                                 # e^(-E/T) = used for probability
        rnum = random.uniform(0,1)                                 # Generating a random floating number b/w 0 & 1
        
        if E < 0 or rnum < exp(e):                                 # Comparing the probability
            current_board = neighbour
            current_cost = neighbour_cost
            
        if (current_cost == 0):                                    # If the solution is found
            return current_board, current_cost, moves1
            break   
    return current_board,current_cost, moves1
            
        

#################################################### MAIN IMPLEMENTATION #######################################################


def main():
    global board_size
    board_size = int(input("Enter the board size:"))
    
    rows , cols = (board_size,board_size)
    board = []                                       

    for i in range(rows):                           # Loop iterating through rows
        col = []
        for j in range(cols):                       # Loop iterating through columns
            col.append("*")                         # Initially storing "-" to show every cell is blocked.
        board.append(col)
    print()
    print("#####################################################################")
    print("########################### INITIAL BOARD ###########################")
    print("#####################################################################")
    print()
    print ("Initial Board after randomization = ")
    print()
    placeQueens(board)
    printBoard(board)
    print()
    print()
    print("#####################################################################")
    print("##################### HILL CLIMBING ALGORITHM #######################")
    print("#####################################################################")
    
    result, moves, h = hillClimbing(board)
    print()
    print("The BEST possible solution is = ")
    print()
    printBoard(result)
    print()
    print("The number of attacking queens in this combination are = ", h)
    print()
    print("The total number of moves (cost) is = ", moves)
    print()
    print()
    print("#####################################################################")
    print("################## SIMULATED ANNEALING ALGORITHM ####################")
    print("#####################################################################")        
    
    result1, h1, moves1 = simulatedAnnealing(board)
    print()
    print("The BEST possible solution is = ")
    print()
    printBoard(result1)
    print()
    print("The number of attacking queens in this combination are = ", h1)
    print()
    print("The total number of moves (cost) is = ", moves1)
    print()
    print()
    print("#####################################################################")
    
    
if __name__ == "__main__":
    main()


Enter the board size:8

#####################################################################
########################### INITIAL BOARD ###########################
#####################################################################

Initial Board after randomization = 

* Q * * * * * * 
Q * * * * * Q * 
* * * Q * * * * 
* * * * * * * * 
* * * * * * * * 
* * Q * * * * Q 
* * * * * * * * 
* * * * Q Q * * 


#####################################################################
##################### HILL CLIMBING ALGORITHM #######################
#####################################################################

The BEST possible solution is = 

Q * * * * * * * 
* * * * * * Q * 
* * * Q * * * * 
* Q * * * * * * 
* * * * * Q * * 
* * * * * * * Q 
* * Q * * * * * 
* * * * Q * * * 

The number of attacking queens in this combination are =  1

The total number of moves (cost) is =  5


#####################################################################
################## SIMULATED ANNE